<a href="https://colab.research.google.com/github/harsha-bsm/Topic-Modelling-using-LDA-and-NMF-on-SamsungM30s-reviewsflipkart/blob/master/POS_TAGGING_NOUNS_and_Adjectives_LDA_SAMSUNGM30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv("samsungdata.csv")
dataoriginal=pd.read_csv("samsungdata.csv")

In [ ]:
data.head()

,Unnamed: 0,Model,rating,user,time_of_review,review,title,user_address,user_type
0,0,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Aryan Patil,2months ago,Bokeh mode is very excellent than any other ha...,Wonderful,Latur District,Certified Buyer
1,1,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,Just awsome product the picture quality is goo...,Great product,North Twenty Four Parganas District,Certified Buyer
2,2,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,4,Suresh Kumar,5months ago,Samsung galaxy m30s is most useful.But Samsung...,Nice product,New Delhi,Certified Buyer
3,3,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,1,Rohit Purba,6months ago,Not worth for me. i buy for my samsung tv this...,Utterly Disappointed,Rajpura,Certified Buyer
4,4,Samsung Galaxy M30s Blue 128 Gb Reviews: Lates...,5,Flipkart Customer,5months ago,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...",Highly recommended,Muzaffarpur,Certified Buyer


In [ ]:
pip install emot 

In [ ]:
####### Apply emoticons and emojis

from emot.emo_unicode import UNICODE_EMO, EMOTICONS # contains english text of emojis and emoticons in dict format 
def convert_emojis(review):
  for x in review:
    if x in UNICODE_EMO.keys():
      review=review.replace(x,"")
  return review

In [ ]:
#converting all basic contracted foms  
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
#removing special characters and tabs ,spaces,new lines etc..
def removecharacters(review):
  review= re.sub('[^A-Za-z0-9]+',' ',review)  #anything exept numbers and alphabets, replace them with space
  review=re.sub(r"\n"," ",review)  #new lines into space
  review=re.sub(r"\t"," ",review)  #tabs into space
  review=re.sub(r"\v"," ",review)  #vertical tab into space
  review=re.sub(r"\s"," ",review)   #all extra spaces into single space
  return review  


In [ ]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step
# ignore words like won't, haven't etc.. We have already taken care of them

stopword= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not","phone","product","mobile","np"])

In [ ]:
from nltk.corpus import stopwords
import nltk

In [ ]:
def removestopwords(review):
  return " ".join([word for word in review.split(" ") if not word in stopword])


In [ ]:
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import ldamodel
import gensim.corpora
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
  topics_cluster=[]
  for topic_idx, topic in enumerate(model.components_):
      message = "Topic #%d: " % topic_idx
      message += ",".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
      topics_cluster.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
  return topics_cluster
  

In [ ]:
  import nltk
  nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def only_nouns(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if pos=="NN"])
  return lis
  


In [ ]:
data["review"]=data["review"].apply(lambda x:only_nouns(x))

## POS TAG NOUNS and ADJECTIVES

In [ ]:
data=pd.read_csv("samsungdata.csv")
data["review"]=data["review"].apply(lambda x:convert_emojis(x))
data["review"]=data["review"].apply(lambda x:x.lower()) 
data["review"]=data["review"].apply(lambda x:decontracted(x))
data["review"]=data["review"].apply(lambda x:removecharacters(x))
data["review"]=pd.DataFrame(data["review"].apply(lambda x:removestopwords(x)))

In [ ]:
def only_nouns_adjectives(review):
  lis= " ".join([each.strip(" ") for each,pos in nltk.pos_tag(review.split()) if (pos=="NN" or pos=="JJ")])
  return lis
data["title"]=data["title"].apply(lambda x:only_nouns_adjectives(x))


### LDA BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)

[['flipkart', 'best', 'value', 'worst'],
 ['super', 'awsome', 'quality', 'fine'],
 ['hours', 'display', 'battery', 'great'],
 ['best', 'camera', 'good', 'bad'],
 ['hangs', 'no', 'thanks', 'flipkart'],
 ['poor', 'excellent', 'good', 'box'],
 ['superb', 'samsung', 'buy', 'smartphone'],
 ['good', 'battery', 'camera', 'performance'],
 ['xcellent', 'awesome', 'money', 'worth'],
 ['nice', 'best', 'wesome', 'samsung']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit_transform(df_bow)
print_top_words(lda,bow.get_feature_names(),4)


[['fine', 'best', 'superb', 'hangs'],
 ['worth', 'battery', 'price', 'smartphone'],
 ['money', 'camera', 'good', 'poor'],
 ['hours', 'buy', 'xcellent', 'price'],
 ['great', 'box', 'samsung', 'flipkart']]

### LDA TFIDF

In [ ]:

tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()
df_bow=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=10,random_state=0)
lda.fit_transform(tfidf_words)
print_top_words(lda,words,5)

[['display', 'performance', 'best', 'battery', 'nicc'],
 ['super', 'awesome', 'flipkart', 'value', 'money'],
 ['thanks', 'suprrrrrrrrr', 'cell', 'tanks', 'perform'],
 ['bad', 'awsm', 'great', 'loving', 'prublam'],
 ['no', 'hangs', 'smooth', 'frequently', 'lovely'],
 ['exellent', 'sm', 'nice', 'isplay', 'life'],
 ['xcellent', 'superb', 'love', 'smartphone', 'flipkart'],
 ['good', 'camera', 'price', 'quality', 'nice'],
 ['backup', 'battery', 'camara', 'worth', 'loved'],
 ['nice', 'wesome', 'best', 'samsung', 'ok']]

#### LDA again

In [ ]:
arr=np.array(print_top_words(lda,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_tfidf=pd.DataFrame(tfidf_words.toarray(),columns=words)
lda = LatentDirichletAllocation(n_components=4,random_state=0)
lda.fit_transform(df_tfidf)
print_top_words(lda,words,4)


[['flipkart', 'backup', 'tanks', 'loving'],
 ['suprrrrrrrrr', 'xcellent', 'sm', 'bad'],
 ['battery', 'worth', 'love', 'frequently'],
 ['nice', 'best', 'value', 'money']]

### NMF BOW

In [ ]:
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(data["review"])

nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['good', 'battery', 'camera', 'display'],
 ['super', 'superb', 'great', 'display'],
 ['samsung', 'buy', 'm30s', 'loved'],
 ['nice', 'delivery', 'thanks', 'awsm'],
 ['quality', 'camera', 'hangs', 'using'],
 ['best', 'price', 'range', 'camera'],
 ['wesome', 'battery', 'money', 'xcellent'],
 ['performance', 'excellent', 'no', 'display'],
 ['use', 'ok', 'best', 'awsome'],
 ['flipkart', 'thank', 'love', 'worst']]

#### NMF again

In [ ]:
arr=np.array(print_top_words(nmf,bow.get_feature_names(),5)).reshape(1,-1)[0]
bow=CountVectorizer(ngram_range=(1,1))
bow_review=bow.fit_transform(arr)
df_bow=pd.DataFrame(bow_review.toarray(),columns=bow.get_feature_names())
nmf = NMF(n_components=5, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(bow_review)
bow_words=bow.get_feature_names()
print_top_words(nmf,bow_words,4)

[['display', 'hours', 'buy', 'worst'],
 ['battery', 'great', 'money', 'good'],
 ['camera', 'xcellent', 'online', 'backup'],
 ['best', 'quality', 'budget', 'hangs'],
 ['performance', 'love', 'samsung', 'better']]

### NMF TFIDF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(data["review"])
words=tfidf.get_feature_names()

In [ ]:
nmf = NMF(n_components=10, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)
print_top_words(nmf,words,6)

[['good', 'one', 'normal', 'choic', 'isplay', 'choice'],
 ['nice', 'awsm', 'phones', 'samsang', 'cell', 'osm'],
 ['super', 'quality', 'camera', 'also', 'performance', 'love'],
 ['wesome', 'outstanding', 'perfomance', 'loving', 'slow', 'gaming'],
 ['best', 'samsung', 'price', 'loved', 'range', 'buy'],
 ['battery', 'camera', 'display', 'backup', 'poor', 'back'],
 ['superb', 'expected', 'great', 'got', 'galaxy', 'tq'],
 ['xcellent', 'flipkart', 'smartphone', 'love', 'excellent', 'thanks'],
 ['money', 'awesome', 'value', 'much', 'worth', 'delivery'],
 ['bad', 'ok', 'thank', 'worst', 'working', 'hangs']]

#### NMF again -------------

In [ ]:
arr=np.array(print_top_words(nmf,words,5)).reshape(1,-1)[0]
tfidf=TfidfVectorizer(ngram_range=(1,1))
tfidf_words=tfidf.fit_transform(arr)
words=tfidf.get_feature_names()
df_lda=pd.DataFrame(tfidf_words.toarray(),columns=words)
nmf = NMF(n_components=4, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf_words)

print_top_words(nmf,words,4)

[['xcellent', 'expected', 'loved', 'bad'],
 ['loving', 'great', 'samsung', 'price'],
 ['excellent', 'samsang', 'bad', 'camera'],
 ['awsm', 'superb', 'slow', 'nice']]

In [ ]:
topics=print_top_words(nmf,bow_words,4)
topic=["topic_"+str(i) for i  in range(0,nmf.n_components)]
df_doc_topic=pd.DataFrame(nmf.fit_transform(df_bow),columns=topic)
dominant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic["dominant_topic"] = dominant_topic
df_doc_topic["review"]=dataoriginal["review"]
df_doc_topic["dominant_topic_keywords"]=[topics[i]  for i in dominant_topic]
df_doc_topic

,topic_0,topic_1,topic_2,topic_3,dominant_topic,review,dominant_topic_keywords
0,0.000000,0.453485,0.000000,0.000000,1,Bokeh mode is very excellent than any other ha...,"[delivery, buy, great, excellent]"
1,0.000000,0.453485,0.000000,0.000000,1,Just awsome product the picture quality is goo...,"[delivery, buy, great, excellent]"
2,0.385215,0.000000,0.000000,0.000000,0,Samsung galaxy m30s is most useful.But Samsung...,"[loved, battery, camera, awsome]"
3,0.000000,0.000000,0.357684,0.000000,2,Not worth for me. i buy for my samsung tv this...,"[bad, good, awsome, backup]"
4,0.000000,0.000000,0.000000,0.357258,3,"BSOLUTELY BEST PHONE, GO FOR THIS PHONELET ME ...","[awsm, look, hangs, display]"
5,0.000000,0.000000,0.000000,0.357258,3,I have realized that this phone is full packag...,"[awsm, look, hangs, display]"
6,0.000000,0.453485,0.000000,0.000000,1,"one of the good phone in this price range , th...","[delivery, buy, great, excellent]"
7,0.000000,0.000000,0.357684,0.000000,2,Nice mobile and it comes good battery back up....,"[bad, good, awsome, backup]"
8,0.000000,0.000000,0.357684,0.000000,2,Been using this phone for more than 4 months n...,"[bad, good, awsome, backup]"
9,0.000000,0.000000,0.357684,0.000000,2,This phone is works excellent so far after (6 ...,"[bad, good, awsome, backup]"
